# Detecting Sentence Entailment

Imports

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

Load the fine-tuned model

In [3]:
model_name = "roberta-large-mnli"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()

model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=Tru

Understand Label Mapping

In [4]:
label_mapping = model.config.id2label
print(label_mapping)

{0: 'CONTRADICTION', 1: 'NEUTRAL', 2: 'ENTAILMENT'}


Prepare Sentence Pairs

In [5]:
premise = "A man is playing a guitar on stage."
hypothesis = "A person is performing music."


Tokenize Sentence Pair

In [6]:
inputs = tokenizer(
    premise,
    hypothesis,
    return_tensors = "pt",
    truncation = True,
    padding = True
)

Run Inference

In [7]:
with torch.no_grad():
  outputs = model(**inputs)

In [8]:
outputs.logits

tensor([[-3.1327, -0.7521,  4.0199]])

Convert Logits → Prediction

In [9]:
logits = outputs.logits
probabilities = torch.softmax(logits, dim = 1)

predicted_class_id = torch.argmax(probabilities, dim = 1).item()
predicted_label = label_mapping[predicted_class_id]

Display Result

In [10]:
confidence = probabilities[0][predicted_class_id].item()

print("Premise:", premise)
print("Hypothesis:", hypothesis)
print("Prediction:", predicted_label)
print("Confidence:", round(confidence, 4))

Premise: A man is playing a guitar on stage.
Hypothesis: A person is performing music.
Prediction: ENTAILMENT
Confidence: 0.9908
